In [49]:
import music21
import pathlib

In [50]:
instruments = {k.lower():v for k,v in vars(music21.instrument).items() if hasattr(v, "bestName")}
instruments

{'instrument': music21.instrument.Instrument,
 'keyboardinstrument': music21.instrument.KeyboardInstrument,
 'piano': music21.instrument.Piano,
 'harpsichord': music21.instrument.Harpsichord,
 'clavichord': music21.instrument.Clavichord,
 'celesta': music21.instrument.Celesta,
 'sampler': music21.instrument.Sampler,
 'electricpiano': music21.instrument.ElectricPiano,
 'organ': music21.instrument.Organ,
 'pipeorgan': music21.instrument.PipeOrgan,
 'electricorgan': music21.instrument.ElectricOrgan,
 'reedorgan': music21.instrument.ReedOrgan,
 'accordion': music21.instrument.Accordion,
 'harmonica': music21.instrument.Harmonica,
 'stringinstrument': music21.instrument.StringInstrument,
 'violin': music21.instrument.Violin,
 'viola': music21.instrument.Viola,
 'violoncello': music21.instrument.Violoncello,
 'contrabass': music21.instrument.Contrabass,
 'harp': music21.instrument.Harp,
 'guitar': music21.instrument.Guitar,
 'acousticguitar': music21.instrument.AcousticGuitar,
 'electricguit

In [58]:
def abc_to_midi(abc_file, destination=None, instrument=None, tempo=None):
    abc_file = pathlib.Path(abc_file)

    if destination is None:
        destination = abc_file.with_suffix(".mid")
        
    score = music21.converter.parse("cooleys.abc")

    if instrument is not None:
        instrument = instruments.get(instrument.lower())
        for p in score.parts:
            p.insert(0, instrument())

    if tempo is not None:
        score.insert(0, music21.tempo.MetronomeMark(number=tempo))

    # Convert to midi
    mf = music21.midi.translate.music21ObjectToMidiFile(score)
    mf.open(destination, "wb")
    mf.write()
    mf.close()

    return destination

In [59]:
abc_to_midi("cooleys.abc", instrument="violin", tempo=160)

PosixPath('cooleys.mid')

In [64]:
import subprocess

def midi_to_mp3(midi_file):
    midi_file = pathlib.Path(midi_file)
    wav_file = midi_file.with_suffix(".wav")
    mp3_file = midi_file.with_suffix(".mp3")

    # Convert to wav
    command = [
        "fluidsynth",
        "-ni",
        "GeneralUser-GS.sf2",
        str(midi_file),
        "-F",
        str(wav_file),
        "-r",
        "44100"
    ]

    subprocess.run(command, check=True, capture_output=True)

    # Convert to mp3
    mp3_file.unlink()
    command = [
        "ffmpeg",
        "-i",
        str(wav_file),
        str(mp3_file)
    ]

    subprocess.run(command, check=True, capture_output=True)

    # Delete wav file
    wav_file.unlink()

    return mp3_file


In [65]:
midi_to_mp3("cooleys.mid")

PosixPath('cooleys.mp3')

In [61]:
def abc_to_mp3(abc_file, **kwargs):
    midi_file = abc_to_midi(abc_file, **kwargs)
    mp3_file = midi_to_mp3(midi_file)

    return mp3_file

In [66]:
abc_to_mp3("cooleys.abc", instrument="flute", tempo=180)

PosixPath('cooleys.mp3')